# Homework 5

**1.** For this question, you need to write a Python implementation for computing the baseline and matrix factorization models for a collaborative filtering application. You must not use the python Surprise or any other existing libraries developed for building recommender system. You can only pandas to load the data and numpy routines to perform matrix computations.    

**(a)** Download the input file named *ratings.csv* and store it in a pandas Dataframe object named data. Reassign the row index so that the users (rows) correspond to John, Mary, Lee, Joe, Kim, and Bob. Note that column headings have already been provided in the input file. 

In [1]:
import pandas as pd

data = pd.read_csv("ratings3.csv")
data.index = ["John", "Mary", "Lee", "Joe", "Kim", "Bob"]
data

,Mission Impossible,Over the Hedge,Back to the Future,Harry Potter,Aladdin
John,5.0,3,4.0,NaN,3
Mary,5.0,4,5.0,5.0,5
Lee,2.0,2,4.0,5.0,1
Joe,3.0,1,1.0,2.0,1
Kim,4.0,5,NaN,2.0,2
Bob,NaN,1,5.0,4.0,2


**(b)** Write a function named baseline that takes the following as its input arguments:

- a user x items ratings matrix (in numpy array format)
- a pair of hyperparameters, lambda1 and lambda2
- a parameter that governs the maximum number of iterations.

The function will compute the baseline model, consisting of 3 parameter values:

- a user_weight vector, w_u 
- an item_weight vector, w_i 
- the model intercept, mu 

The function initializes w_u and w_i to be vectors of 0s and then interatively updates the weights along with that for mu using the update formula shown in the lecture notes. To ensure its correctness, the function should display the updated values for w_u, w_i, and mu in each iteration. You will then apply the function to predict the missing values in the input ratings matrix.

In [2]:
import numpy as np

def baseline(matrix, lambda1, lambda2, maxiter):
    
    # Initialize the model parameters w_u, w_i, and mu
    w_u, w_i = [], []
    for a in range(0, matrix.shape[0]):
        w_u.append(0)
        w_i.append(0)
        
    mu_sum, mu_count = 0, 0
    for row in matrix:
        for val in row:
            if str(val) != 'nan':
                mu_sum += int(val)
                mu_count += 1
                
    mu = mu_sum / mu_count
    
    # Iterates for the specified number of times
    for i in range(0,maxiter):
        
        # Calculates and updates the w_u for each user (row)
        for j in range(0,matrix.shape[0]):
            row_sum, row_count, w_i_index = 0, 0, 0
            for row_val in data.iloc[j]:
                if str(row_val) != "nan":
                    row_sum += (row_val - w_i[w_i_index] - mu)
                    row_count += 1
                w_i_index += 1
            w_u[j] = row_sum / row_count
            
        # Calculates and updates the w_i for each item (column)
        for j in range(0,matrix.shape[1]):
            col_sum, col_count, w_u_index = 0, 0, 0
            for col_val in data.iloc[:,j]:
                if str(col_val) != "nan":
                    col_sum += (col_val - w_u[w_u_index] - mu)
                    col_count += 1
                w_u_index += 1
            w_i[j] = col_sum / col_count 
            
        # Display the updated model parameters
        print('Iteration ', i)
        print('   w_u =', w_u)
        print('   w_i =', w_i)
        print('   mu  =', mu)

    return (w_u, w_i, mu)

In [3]:
# Apply the model to the ratings data. Set lambda1=lambda2=0 and maxiter = 5
w_u, w_i, mu = baseline(data.values, 0, 0, 5)

# Predicts and updates the ratings for each missing rating
predicted = data.values
for row in range(0,predicted.shape[0]):
    for col in range(0,predicted.shape[1]):
        if np.isnan(predicted[row,col]):
            predicted[row,col] = round(mu + w_u[row] + w_i[col],2)
            
print('Predicted rating for John on Harry Potter =',  predicted[0,3])
print('Predicted rating for Kim on Back to the Future =', predicted[4,2])
print('Predicted rating for Bob on Mission Impossible =', predicted[5,0])

Iteration  0
   w_u = [0.5648148148148149, 1.614814814814815, -0.38518518518518513, -1.585185185185185, 0.06481481481481488, -0.18518518518518512]
   w_i = [0.5600000000000002, -0.5333333333333333, 0.6100000000000001, 0.51, -0.8666666666666666, 0]
   mu  = 3.185185185185185
Iteration  1
   w_u = [0.6223148148148147, 1.5588148148148147, -0.4411851851851852, -1.641185185185185, 0.14731481481481484, -0.11518518518518517]
   w_i = [0.5656, -0.5403333333333331, 0.6181000000000001, 0.5131, -0.8736666666666665, 0]
   mu  = 3.185185185185185
Iteration  2
   w_u = [0.6223898148148147, 1.558254814814815, -0.4417451851851852, -1.6417451851851852, 0.1486398148148147, -0.1144851851851853]
   w_i = [0.5656559999999999, -0.5404033333333332, 0.6182810000000002, 0.5130310000000001, -0.8737366666666665, 0]
   mu  = 3.185185185185185
Iteration  3
   w_u = [0.6223655648148149, 1.5582492148148148, -0.4417507851851852, -1.6417507851851851, 0.14867806481481471, -0.11447818518518538]
   w_i = [0.5656565600000

**(b)** Write a function named MF that employs the matrix factorization approach to predict the missing values in the ratings matrix. The function takes the following as its input arguments:

- an incomplete user x items ratings matrix 
- the number of latent features, k 
- the maximum number of iterations, maxiter

The function should return a pair of matrices, U and M, where U is the user x latent factor matrix and M is the item x latent factor matrix. First, the function initializes the input matrix such that the missing ratings (NaNs) are replaced by 0. It then initializes the matrices U and M to be all 1s. The function then interatively updates the matrices M and U using the update formula shown in the lecture notes. It will use the estimated U and M matrices to re-compute the missing ratings in the input matrix. You will apply the MF function to predict the missing ratings in the input  matrix, assuming the number of latent factors is equal to 2.

In [4]:
def MF(matrix, k, maxiter):
    
    # Initialize the missing ratings in matrix to 0 and stores the indices with nan
    nan_indices = []
    for row in range(0,matrix.shape[0]):
        for col in range(0,matrix.shape[1]):
            if np.isnan(matrix[row,col]) == True:
                nan_indices.append([row,col])
                matrix[row,col] = 0
    
    # Initialize the matrices U and M to 1s. Size of U is #rows x k, size of M is $columns x k
    U = np.ones((matrix.shape[0],k))
    M = np.ones((matrix.shape[1],k))

    # Iterates the specified number of times
    for i in range(maxiter):
        
        # Update M
        mNumerator = np.dot(matrix.T, U)
        mDenom = np.dot(np.dot(M, U.T), U)
        M = np.multiply(M,np.divide(mNumerator,mDenom))

        # Update U
        uNumerator = np.dot(matrix, M)
        uDenom = np.dot(np.dot(U, M.T), M)
        U = np.multiply(U,np.divide(uNumerator,uDenom))

        # Update the missing ratings in matrix with the predicted values U x M^T 
        UM = np.dot(U,M.T)
        
        # Updates the missing values in the matrix
        for index in nan_indices:
            matrix[index[0],index[1]] = UM[index[0],index[1]]
                
    return U, M

In [5]:
# Apply the model to the ratings data. Set k=2 and maxiter = 500
U, M = MF(data.values, 2, 500)

# Calculates the UM^T and updates the missing values 
UM = np.dot(U,M.T)
predicted = data.values
for row in range(0,predicted.shape[0]):
    for col in range(0,predicted.shape[1]):
        if np.isnan(predicted[row,col]) == True:
            predicted[row,col] = UM[row,col]

print('Predicted rating for John on Harry Potter =', predicted[0][3])
print('Predicted rating for Kim on Back to the Future =', predicted[4][2])
print('Predicted rating for Bob on Mission Impossible =', predicted[5][0])

Predicted rating for John on Harry Potter = 4.313339602440373
Predicted rating for Kim on Back to the Future = 3.8231074988350353
Predicted rating for Bob on Mission Impossible = 3.7047639373213035


---
**3.** Consider the student enrollment database below, which contains the following tables:

<img src="student.png">

Draw the tables obtained as query result for each SQL statement below. 

**(a)** 

    SELECT S.Name

    FROM Professor P, Student S, Teaching T, Transcript R

    WHERE P.Id = T.ProfId and S.Id = R.StudId and T.CrsCode = R.CrsCode and T.Semester = R.Semester

            and P.Name = `Mary Doe';
    
**Solution:** You can use the template below to display the query result. Make sure you remove the unnecessary rows and columns and rename the column headings.

| Name | 
|:-----------|
| Bart Simpson | 



**(b)**

    SELECT S.Name, COUNT(T.CrsCode)

    FROM Student S, Transcript T  

    WHERE S.Id=T.StudId

    GROUP BY T.StudId;
    
**Solution:**

| Name | COUNT(T.CrsCode) | ColumnName       
|:-----------|:-----------|:------
| Homer Simpson | 2 | Value
| John Dow | 3 | Value
| Joe Blow | 3 | Value
| Joseph Public | 3 | Value
| Bart Simpson | 2 | Value


**(c)**

    SELECT DISTINCT C.CrsName, T1.Semester

    FROM Teaching T1, Teaching T2, Course C

    WHERE T1.ProfId <> T2.ProfId AND T1.CrsCode = T2.CrsCode

            AND T1.CrsCode = C.CrsCode AND T1.Semester = T2.Semester;
    
**Solution:**

| CrsName | Semester | ColumnName       
|:-----------|:-----------|:------
| Market Analysis | F1994 | Value
| Market Analysis | F1997 | Value
| Value | Value | Value
| Value | Value | Value
| Value | Value | Value


**(d)**

    SELECT DISTINCT S.Name

    FROM Student S

    WHERE EXISTS (SELECT * FROM Transcript T 
    
                  WHERE S.Id = T.StudId AND T.CrsCode LIKE '%CS%');
                  
**Solution:**

| Name | ColumnName | ColumnName       
|:-----------|:-----------|:------
| Bart Simpson | Value | Value
| Joe Blow | Value | Value
| Homer Simpson | Value | Value
| Value | Value | Value
| Value | Value | Value


---
**4.**  Consider the following schema for an electronic health record database for a healthcare provider. Primary and foreign key constraints are also listed for each table (if available).

    Patient(ID, Name)   Primary key = ID

    Physician(ID, Name, Specialty)   Primary key = ID
    
    Insurance(PolicyNo, PatientID, Company)   Primary key = PolicyNo
        Insurance(PatientID) references Patient(ID)

    Visit(ID, PatientID, PhysicianID, ReasonForVisit)   
    
        Primary key = ID

        Visit(PatientID) references Patient(ID)

        Visit(PhysicianID) references Physician(ID)

Assume that some patients can be in the Patient table but not in the Visit table if he/she has not visited the provider (e.g., a newly registered patient). You can also assume that only patients with insurance are recorded in the Insurance table.

Express each of the following query in SQL.

**(a)** Find the names of Physicians who have treated the patient John Doe during one of his visits to the provider. 

**Solution:**
SELECT D.Name
FROM Patient P, Physician D, Visit V
WHERE P.Name = "John Doe" AND P.ID = V.PatientID AND D.ID = V.PhysicianID

**(b)** Find the unique IDs and names of patients who visited the healthcare provider and have insurance. 

**Solution:** 

SELECT DISTINCT P.ID, P.Name
FROM Patient P, Insurance I, Visit V
WHERE P.ID = I.PatientID AND P.ID = V.PatientID


**(c)** Find the unique IDs and names of patients who visited the healthcare provider but have no insurance. 

**Solution:** 
SELECT DISTINCT P.ID, P.Name
FROM Patient P, Insurance I, Visit V
WHERE P.ID = V.PatientID AND NOT EXIST(SELECT * FROM I WHERE P.ID = I.PATIENTID)

**(d)** Find the IDs and names of patients who ahve been treated by more than one physicians.

**Solution:** 
SELECT P.ID, P.Name
FROM Paitent P, Visit V1, Visit V2
WHERE V1.PhysicianID != V2.PhysicianID AND V1.PatientID = V2.PatientID AND P.ID = V1.PatientID;

**(e)** For each physician, count the distinct number of patients they have treated (i.e., met during one of the patients' visits). The query result should contain the physician ID, name, and number of treated patients.

**Solution:**
SELECT D.ID, D.Name, COUNT(DISTINCT V.PatientID)
FROM Physician D, Visit V
WHERE D.ID = V.PhysicianID
GROUP BY D.ID

**5.** Consider the following schema for a bike sharing database:

     Customer(ID, Name, Address)    PRIMARY KEY = ID
     
     Bike(ID, Type)     PRIMARY KEY = ID
     
     Rental(CustomerID, BikeID, CheckoutDate, ReturnDate)
     
         PRIMARY KEY = (CustomerID, BikeID, CheckoutDate)

         Rental(CustomerID) REFERENCES Customer(ID)
         
         Rental(BikeID) REFERENCES Bike(ID)
         
Assume the number of rows in each table are as follows: Customer (1000 rows), Bike (500 rows), and Rental (10000 rows). For each SQL query below, list the attribute names and state the minimum and maximum number of possible rows in the query result. If the query result returns two attributes with the same name, make sure you indicate the name of the table from which the column is obtained, e.g., Customer.ID or Bike.ID.

**(a)** 

        SELECT *

        FROM Customer C, Rental R;
        
        WHERE C.ID = R.CustomerID AND C.Name = 'John Doe';

**Solution:**

Attributes: C.ID, Name, Address, CustomerID, BikeID, CheckoutDate, ReturnDate

Number of rows:  Minimum = 0   Maximum = 10,000

**(b)** 

        SELECT C.Name, B.Type
        
        FROM Customer C, Bike B, Rental R
        
        WHERE C.ID = R.CustomerID AND B.ID = R.BikeID;


**Solution:**

Attributes: C.Name, B.Type

Number of rows: Minimum = 10,000     Maximum = 10,000

**(c)** 

        SELECT DISTINCT B.ID, B.Type
        
        FROM Bike B, Rental R
        
        WHERE B.ID = R.BikeID;


**Solution:**

Attributes: B.ID, B.Type

Number of rows:  Minimum = 1   Maximum = 500


**(d)** 

        SELECT *

        FROM Customer, Bike;

**Solution:**

Attributes: Customer.ID, Name, Address, Bike.ID, Type

Number of rows:  Minimum = 500,000   Maximum = 500,000       

**(e)**  Assuming there are 3 types of bicycles in the database:

        SELECT C.ID, B.Type, COUNT(*) AS Num

        FROM Customer C, Bike B, Rental R
        
        WHERE C.ID = R.CustomerID AND B.ID = R.BikeID
        
        GROUP BY C.ID, B.Type

**Solution:**

Attributes: C.ID, B.Type, Num 

Number of rows:  Minimum = 1   Maximum = 3,000